In [1]:
import os

In [2]:
%pwd

'/Users/sameerraj/Desktop/Machine_learning_python/End-to-End-Text-summarization-project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/sameerraj/Desktop/Machine_learning_python/End-to-End-Text-summarization-project'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [7]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories

In [8]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)->DataTransformationConfig:
        config=self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config

In [11]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

In [12]:
class DataTransformation:
    def __init__(self,config: DataTransformationConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_example_to_features(self,example_batch):
        input_encodings=self.tokenizer(example_batch['dialogue'], max_length = 1024, truncation =True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings=self.tokenizer(example_batch['summary'],max_length=128, truncation=True)

            return {
                'input_ids': input_encodings['input_ids'],
                'attention_mask': input_encodings['attention_mask'],
                'labels': target_encodings['input_ids']
            }
        
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [13]:
try:
    config=ConfigurationManager()
    data_transformation_config=config.get_data_transformation_config()
    data_transformation=DataTransformation(config=data_transformation_config)
    data_transformation.config()
except Exception as e:
    raise e

[2023-11-21 10:16:23,403: INFO: common: yaml file: /Users/sameerraj/Desktop/Machine_learning_python/End-to-End-Text-summarization-project/config/config.yaml loaded successfully]
[2023-11-21 10:16:23,407: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-21 10:16:23,408: INFO: common: created directory at: artifacts]
[2023-11-21 10:16:23,409: INFO: common: created directory at: artifacts/data_transformation]


tokenizer_config.json: 100%|██████████| 88.0/88.0 [00:00<00:00, 148kB/s]
config.json: 100%|██████████| 1.12k/1.12k [00:00<00:00, 5.21MB/s]
spiece.model: 100%|██████████| 1.91M/1.91M [00:01<00:00, 1.73MB/s]
special_tokens_map.json: 100%|██████████| 65.0/65.0 [00:00<00:00, 215kB/s]


ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.